## Preparation

##### Import Functions

In [ ]:
from Class.Helper import Helper as Helper
from Scripts.LoadCSV import LoadCSV as LoadCSV
from Scripts.Fuzzify import Fuzzify as Fuzzify
from Scripts.ValueTest import ValueTest as ValueTest
from Scripts.OptimizeBruteForce import OptimizeBruteForce as OptimizeBruteForce
from Scripts.OptimizeBruteForceKFold import OptimizeBruteForceKFold as OptimizeBruteForceKFold
import warnings
warnings.simplefilter("ignore")

## Experiment 1 -- Gauss Number & Gauss Style & Gauss Adjustment

In [ ]:
import Settings.GeneralSettings as generalSettings
from Settings.SettingsPimaIndiansDiabetes import Settings as Settings

helper = Helper()

for gauss in [3, 5, 7, 9, 11]:
    for style in ["Gaussian Progressive", "Gaussian Equal"]:
        for adjustment in [-1, -2]:
            generalSettings.gausses = gauss
            generalSettings.style = style
            generalSettings.adjustment_value = adjustment
            settings = Settings(generalSettings)        

            loadCSV = LoadCSV()
            samples_stats, train_stats, test_stats, train_samples = loadCSV.worker(settings)

            fuzzify = Fuzzify()
            changed_decisions, features_number_after_reduct, implicants_number, fuzzify_parameters = fuzzify.worker(settings, settings.adjustment_value)

            fuzzification_data = [settings.dataset_name, settings.style, settings.gausses, settings.adjustment, samples_stats, train_stats, test_stats, changed_decisions, round(changed_decisions / train_samples, 2), implicants_number, settings.feature_numbers, features_number_after_reduct]
            helper.saveFuzzificationStats(fuzzification_data)

            valueTest = ValueTest(settings, settings.s_function_width, settings.is_training)
            valueTest.sOptymalizationWorker(settings, settings.default_s_value, settings.show_results)

            valueTest = ValueTest(settings, settings.s_function_width, not settings.is_training)
            valueTest.sOptymalizationWorker(settings, settings.default_s_value, settings.show_results)

## Experiment 2 -- S-Function Optymalization

In [ ]:
import Settings.GeneralSettings as generalSettings
from Settings.SettingsPimaIndiansDiabetes import Settings as Settings
settings = Settings(generalSettings)

optimizeBruteForceKFold = OptimizeBruteForceKFold(settings, settings.s_function_width)
valueTest = ValueTest(settings, settings.s_function_width, False)

best_s_function_center, mean_s_function_center, changed_decisions, features_number_after_reduct, implicants_number = optimizeBruteForceKFold.worker(settings, settings.constraints, settings.s_function_width, settings.n_folds)


valueTest.sOptymalizationWorker(settings, best_s_function_center, "Test Value - K-Fold - Best Function Center")
threshold = optimizeBruteForceKFold.thresholdWorker(settings, best_s_function_center, settings.s_function_width)
valueTest.thresholdWorker(settings, best_s_function_center, threshold)

valueTest.sOptymalizationWorker(settings, mean_s_function_center, "Test Value - K-Fold - Average Function Center")
threshold = optimizeBruteForceKFold.thresholdWorker(settings, mean_s_function_center, settings.s_function_width)
valueTest.thresholdWorker(settings, mean_s_function_center, threshold)